In [93]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Random Forest').getOrCreate()
import pandas as pd
import numpy as np 

In [94]:
from ipynb.fs.full.Knn import *

In [95]:
df4

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,formerly smoked,0


In [96]:
dfRf = df4[['id','gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','smoking_status','stroke']]

In [97]:
dfRf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 439.3+ KB


In [98]:
dfRf = dfRf.replace(to_replace =['Female', 'Male', 'Other'], value = [1,0,2])
dfRf = dfRf.replace(to_replace =['Yes', 'No'], value = [1,0])
dfRf = dfRf.replace(to_replace =['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked'], value = [1,0,2,3,4])
dfRf = dfRf.replace(to_replace =['Urban', 'Rural'], value = [1,0])
dfRf = dfRf.replace(to_replace =['never smoked', 'smokes', 'formerly smoked'], value = [0,1,2])

dfRf['gender'] = dfRf['gender'].astype(str).astype(int)
dfRf['ever_married'] = dfRf['ever_married'].astype(str).astype(int)
dfRf['work_type'] = dfRf['work_type'].astype(str).astype(int)
dfRf['Residence_type'] = dfRf['Residence_type'].astype(str).astype(int)
dfRf['smoking_status'] = dfRf['smoking_status'].astype(str).astype(int)

In [99]:
dfRf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int64  
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int64  
 6   work_type          5110 non-null   int64  
 7   Residence_type     5110 non-null   int64  
 8   avg_glucose_level  5110 non-null   float64
 9   smoking_status     5110 non-null   int64  
 10  stroke             5110 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 439.3 KB


In [100]:
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import RandomForestClassifier

In [102]:
dfRf.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status,stroke
0,9046,0,67.0,0,1,1,1,1,228.69,2,1
1,51676,1,61.0,0,0,1,0,0,202.21,0,1
2,31112,0,80.0,0,1,1,1,0,105.92,0,1
3,60182,1,49.0,0,0,1,1,1,171.23,1,1
4,1665,1,79.0,1,0,1,0,0,174.12,0,1


In [104]:
predictors = dfRf.drop(['stroke'], axis=1)

In [105]:
predictors.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status
0,9046,0,67.0,0,1,1,1,1,228.69,2
1,51676,1,61.0,0,0,1,0,0,202.21,0
2,31112,0,80.0,0,1,1,1,0,105.92,0
3,60182,1,49.0,0,0,1,1,1,171.23,1
4,1665,1,79.0,1,0,1,0,0,174.12,0


In [108]:
stroke = dfRf['stroke']

In [109]:
x_train, x_val, y_train, y_val = train_test_split(predictors, stroke, test_size = 0.25, random_state = 0)

In [110]:
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)

filename = 'model.sav'        #trained model save in this file
pickle.dump(randomforest, open(filename, 'wb'))

In [111]:
def prediction_model(id, gender, age, hypertension, heart_disease, ever_married, work_type,	Residence_type, avg_glucose_level, smoking_status):
    
    x= [[id, gender, age, hypertension, heart_disease, ever_married, work_type,	Residence_type, avg_glucose_level, smoking_status]]
    randomforest=pickle.load(open('model.sav','rb'))
    predictions =randomforest.predict(x)
    print(predictions)

In [118]:
prediction_model(120,219,0,1,158,0,1.6,1,0,2)

[0]


In [120]:
prediction_model(138,166,0,0,125,1,3.6,1,1,2)

[0]
